<a href="https://colab.research.google.com/github/Sohila912/CNN_fashion_mnist/blob/main/fashion_mnist_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.17.1


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images, test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
#since the images are in grayscale i can scale them to be from 0 to 1 only
train_images = train_images/255.0
test_images = test_images/255.0


In [7]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          #The difference between hp.Choice and hp.Int
          #hp.Choice: chooses from a given set while hp.Int goes on integers with a start, end and step
          filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
          kernel_size = hp.Choice('conv_1_kernel' , values = [3,5]), #Controls the size of the sliding window (e.g., 3x3 or 5x5) used for convolution. Larger kernels capture more spatial information but require more computation.
          activation = 'relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
          kernel_size = hp.Choice('conv_2_kernel' , values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(), #Converts the 2D feature maps into a 1D vector, Prepares the data for fully connected (Dense) layers by linearizing the multi-dimensional feature maps.
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
          activation = 'relu'
      ),
      keras.layers.Dense(10,activation = 'softmax')
  ])

  model.compile (optimizer = keras.optimizers.Adam(hp.Choice('learning_rate' , values = [1e-2, 1e-3])),
                 loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
#Tuner searches for the best learning rate and the number of layers
tuner_search = RandomSearch(build_model,
                            objective= 'val_accuracy',
                            max_trials=5,directory = 'output', project_name = "Mnist Fashion")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
tuner_search.search(train_images,train_labels,epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.9089999794960022

Best val_accuracy So Far: 0.9089999794960022
Total elapsed time: 00h 02m 56s


In [13]:
model = tuner_search.get_best_models(num_models= 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 96)          │           2,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 20, 20, 48)          │         115,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 19200)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │         614,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 732,506 (2.79 MB)

 Trainable params: 732,506 (2.79 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9243 - loss: 0.2042 - val_accuracy: 0.9053 - val_loss: 0.2851
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9400 - loss: 0.1640 - val_accuracy: 0.9062 - val_loss: 0.2804
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9510 - loss: 0.1347 - val_accuracy: 0.9073 - val_loss: 0.2779
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9575 - loss: 0.1127 - val_accuracy: 0.9093 - val_loss: 0.3301
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9652 - loss: 0.0934 - val_accuracy: 0.9097 - val_loss: 0.3309
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9706 - loss: 0.0757 - val_accuracy: 0.9075 - val_loss: 0.3631
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9769 - loss: 0.0627 - val_accuracy: 0.9090 - val_loss: 0.4036


In [16]:
model.fit(train_images,train_labels,epochs=20,validation_split=0.1,initial_epoch=10)

Epoch 11/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9799 - loss: 0.0540 - val_accuracy: 0.9095 - val_loss: 0.4266
Epoch 12/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9821 - loss: 0.0486 - val_accuracy: 0.9073 - val_loss: 0.4542
Epoch 13/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9859 - loss: 0.0408 - val_accuracy: 0.9038 - val_loss: 0.5262
Epoch 14/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9865 - loss: 0.0366 - val_accuracy: 0.9082 - val_loss: 0.5347
Epoch 15/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9867 - loss: 0.0376 - val_accuracy: 0.9052 - val_loss: 0.5674
Epoch 16/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9891 - loss: 0.0283 - val_accuracy: 0.9088 - val_loss: 0.5928
Epoch 17/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9907 - loss: 0.0271 - val_accuracy: 0.9067 - val_loss: 0.6598
Epoch 18/20
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9911 - loss: